In [8]:
import pandas as pd
import numpy as np
import networkx as nx

# LncRNA- Disease Dataset
Total no. of lncRNAs= 240

Total no of Diseases= 412

Total no of miRNAs = 495

Total no. of lnc-Dis interaction = 2697

In [11]:
lncrnas = pd.read_csv('lncRNAs.csv')
lncrnas = list(lncrnas['lncrna'])

diseases = pd.read_csv('diseases.csv')
diseases =list(diseases['disease'])

mirnas = pd.read_csv('miRNAs.csv')
mirnas = list(mirnas['mirna'])

print("Lnc=",len(lncrnas),"miRNA=",len(mirnas),"Diseases=",len(diseases))

Lnc= 240 miRNA= 495 Diseases= 412


In [12]:
def name2id(names,label):
    name2id_dic = {}
    n=1
    for name in names:
        name2id_dic[name]=label+str(n)
        n+=1
    return name2id_dic

def id2name(ids):
    pass



In [13]:
lnc2id = name2id(lncrnas,'L')
mir2id = name2id(mirnas, 'M')
dis2id = name2id(diseases,'D')

lncIdx = [lnc2id[lnc] for lnc in lncrnas]
mirIdx = [mir2id[mir] for mir in mirnas]
disIdx = [dis2id[dis] for dis in diseases]

#lncIdx[:10],mirIdx[:10],disIdx[:10]

In [14]:
lnc_dis_df = pd.read_csv('lncRNA-disease.csv')
lnc_dis_df.shape

(239, 412)

In [17]:
temp ={}
for dis in disIdx:
    temp[dis]=lnc_dis_df[dis].values
lnc_dis_dict={}

n=0
for lnc in lncIdx:
    t={}
    for dis in disIdx:
        t[dis]=temp[dis][n]
    lnc_dis_dict[lnc]=t
    n+=1

#temp['D2'].shape,len(lnc_dis_dict['L45'])
#print(lnc_dis_dict['L2'])
    
#lnc_dis_df.head(2)
        

KeyError: 'D1'

In [ ]:
lnc_dis_adj_list = []
for lnc in lncIdx:
    for dis in disIdx:
        lnc_dis_adj_list.append((lnc,dis,lnc_dis_dict[lnc][dis]))
len(lnc_dis_adj_list)

In [ ]:
pos=[]
pos_lnc =[]
pos_dis =[]
for lnc in lncIdx:
    for dis in disIdx:
        if lnc_dis_dict[lnc][dis]>0:
            pos.append((lnc,dis,1))
            if lnc not in pos_lnc: pos_lnc.append(lnc)
            if dis not in pos_dis: pos_dis.append(dis)
print (len(pos_lnc),len(pos_dis),len(pos))


In [ ]:
neg_all=[]
for lnc in lncIdx:
    for dis in disIdx:
        if lnc_dis_dict[lnc][dis]==0:
            neg_all.append((lnc,dis,0))

print(len(neg_all))

In [ ]:
import random
random.seed(1024)
neg = random.sample(neg_all,2697)

In [ ]:
len(set([l[0] for l in neg])),len(set([l[1] for l in neg])),len(neg)

In [ ]:
random.shuffle(pos)
random.shuffle(neg)

In [ ]:
total = len(pos)
train = pos[:int(total*0.8)]
train += neg[:int(total*0.8)]
test = pos[int(total*0.8):]
test += neg[int(total*0.8):]
len(train),len(test)

In [ ]:
random.shuffle(train)
random.shuffle(test)

In [ ]:
fp=open('X_train.txt','w')
[fp.write(pair[0]+','+pair[1]+'\n') for pair in train]
fp.close()

fp=open('X_test.txt','w')
[fp.write(pair[0]+','+pair[1]+'\n') for pair in test]
fp.close()

fp=open('y_train.txt','w')
[fp.write(str(pair[2])+'\n') for pair in train]
fp.close()

fp=open('y_test.txt','w')
[fp.write(str(pair[2])+'\n') for pair in test]
fp.close()

# LDA: LncRNA-Disease Network

In [ ]:
lnc_dis_adj_list = []
s=0
fp=open('lda.edgelist','w')
for lnc in lncIdx:
    for dis in disIdx:
        s+=int(bool(fp.write(lnc+'\t'+dis+'\t'+str(lnc_dis_dict[lnc][dis])+'\n')))
print(s)
fp.close()


In [ ]:
lda_graph = nx.read_weighted_edgelist('lda.edgelist',delimiter="\t")
lda_adj = nx.adjacency_matrix(lda_graph)
lda_adj.shape

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

nx.draw(lda_graph,node_color=colors, cmap = plt.get_cmap('rainbow'), with_labels=True,pos=nx.spring_layout (lda_graph))

In [ ]:
print("Before:",lda_graph.number_of_nodes(),lda_graph.number_of_edges())
for edge in lda_graph.edges:
    if lda_graph.get_edge_data(edge[0],edge[1])['weight']<=0.0:
        lda_graph.remove_edge(edge[0],edge[1])
print("After:",lda_graph.number_of_nodes(),lda_graph.number_of_edges())

In [ ]:
from node2vec import Node2Vec

In [ ]:
node2vec = Node2Vec(lda_graph, dimensions=64, walk_length=10, num_walks=100, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [ ]:
ld_l_vec={}
for lnc in lncIdx:
    ld_l_vec[lnc]=model.wv.get_vector(lnc)

ld_d_vec={}
for dis in disIdx:
    ld_d_vec[dis]=model.wv.get_vector(dis)

len(ld_l_vec),len(ld_d_vec)

In [ ]:
X_train_ld =[]
for ld in train:
    ld_vec = np.concatenate((ld_l_vec[ld[0]],ld_d_vec[ld[1]]))
    X_train_ld.append(ld_vec)
X_train_ld = np.array(X_train_ld)

X_test_ld =[]
for ld in test:
    ld_vec = np.concatenate((ld_l_vec[ld[0]],ld_d_vec[ld[1]]))
    X_test_ld.append(ld_vec)
X_test_ld = np.array(X_test_ld)

X_train_ld.shape, X_test_ld.shape

In [ ]:
np.save('X_train_ld.npy',X_train_ld)
np.save('X_test_ld.npy',X_test_ld)

# LMS: lncRNA-miRNA-lncRNA interaction network

In [ ]:
lnc_mir_df = pd.read_csv('lncRNA-miRNA.csv')
lnc_mir_df.shape

In [ ]:
#lnc_mir_df.values[0][1:]

In [ ]:
lnc_mir_values=lnc_mir_df.values
lnc_mir_dic={}
for i in range(lnc_mir_df.shape[0]):
    lnc = lnc_mir_values[i][0]
    lnc_mir_dic[lnc]={mirIdx[j-1]:lnc_mir_values[i][j] for j in range(1,lnc_mir_df.shape[1])}
#lnc_mir_dic['L1']['M2']

In [ ]:
count=0
pos_lnc_mir =[]
pos_mir =[]
for lnc in lncIdx:
    for mir in mirIdx:
        if lnc_mir_dic[lnc][mir]>0:
            count+=1
            if lnc not in pos_lnc_mir:
                pos_lnc_mir.append(lnc)
            if mir not in pos_mir:
                pos_mir.append(mir)
print(len(pos_lnc_mir),len(pos_mir),count)

In [ ]:
fp = open('lnc_mir.edgelist','w')
for lnc in lncIdx:
    for mir in mirIdx:
        fp.write(lnc+'\t'+mir+'\t'+str(lnc_mir_dic[lnc][mir])+'\n')
fp.close()


In [ ]:
lnc_mir_graph = nx.read_weighted_edgelist('lnc_mir.edgelist',delimiter='\t')
#nx.draw(lnc_mir_graph, cmap = plt.get_cmap('rainbow'), with_labels=True,pos=nx.spring_layout (lnc_mir_graph))

In [ ]:
lnc_lnc_df = pd.read_csv('lncRNA-lncRNA.csv')
lnc_lnc_values=lnc_lnc_df.values
lnc_lnc_dic={}
for i in range(lnc_lnc_df.shape[0]):
    lnc = lnc_lnc_df.values[i][0]
    lnc_lnc_dic[lnc]={lncIdx[j-1]:lnc_lnc_values[i][j] for j in range(1,lnc_lnc_df.shape[1])}


In [ ]:
fp = open('lnc_lnc.edgelist','w')
for lnc1 in lncIdx:
    for lnc2 in lncIdx:
        fp.write(lnc1+'\t'+lnc2+'\t'+str(lnc_lnc_dic[lnc1][lnc2])+'\n')
fp.close()

In [ ]:
lnc_lnc_edges = open('lnc_lnc.edgelist').read()
lnc_mir_edges = open('lnc_mir.edgelist').read()
fp= open('lnc_mir_lnc.edgelist','w')
fp.write(lnc_lnc_edges)
fp.write(lnc_mir_edges)
fp.close()

In [ ]:
lml_graph = nx.read_weighted_edgelist('lnc_mir_lnc.edgelist',delimiter='\t')

In [ ]:
print("Before:",lml_graph.number_of_nodes(),lml_graph.number_of_edges())
for edge in lml_graph.edges:
    if lml_graph.get_edge_data(edge[0],edge[1])['weight']<=0.0:
        lml_graph.remove_edge(edge[0],edge[1])
print("After:",lml_graph.number_of_nodes(),lml_graph.number_of_edges())

In [ ]:
node2vec = Node2Vec(lml_graph, dimensions=64, walk_length=10, num_walks=100, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [ ]:
lm_l_vec={}
for lnc in lncIdx:
    lm_l_vec[lnc]=model.wv.get_vector(lnc)

In [ ]:
lm_l_vec['L2'].shape

# DMS: Disease- miRNA- Disease Network

In [ ]:
dis_mir_df = pd.read_csv ('miRNA-disease.csv', header=None)
dis_mir_df = dis_mir_df.transpose()
dis_mir_df.shape

In [ ]:
dis_mir_values=dis_mir_df.values
dis_mir_dic={}
for i in range(dis_mir_df.shape[0]):
    dis = disIdx[i]
    dis_mir_dic[dis]={mirIdx[j]:dis_mir_values[i][j] for j in range(dis_mir_df.shape[1])}


In [ ]:
dis_dis_df = pd.read_csv ('disease-disease.csv',sep='\t', header=None)
dis_dis_df.shape

In [ ]:
dis_dis_values=dis_dis_df.values
dis_dis_dic={}
for i in range(dis_dis_df.shape[0]):
    dis = disIdx[i]
    dis_dis_dic[dis]={disIdx[j]:dis_dis_values[i][j] for j in range(dis_dis_df.shape[1])}


In [ ]:
dis_mir_edges=''
for dis in disIdx:
    for mir in mirIdx:
        dis_mir_edges+=dis+'\t'+mir+'\t'+str(dis_mir_dic[dis][mir])+'\n'

dis_dis_edges =''
for dis in disIdx:
    for dis in disIdx:
        dis_dis_edges+=dis+'\t'+dis+'\t'+str(dis_dis_dic[dis][dis])+'\n'

fp = open('dis_mir_dis.edgelist','w')
fp.write(dis_mir_edges+dis_dis_edges)
fp.close()

In [ ]:
dmd_graph = nx.read_weighted_edgelist('dis_mir_dis.edgelist',delimiter='\t')

In [ ]:
print("Before:",dmd_graph.number_of_nodes(),dmd_graph.number_of_edges())
for edge in dmd_graph.edges:
    if dmd_graph.get_edge_data(edge[0],edge[1])['weight']<=0.0:
        dmd_graph.remove_edge(edge[0],edge[1])
print("After:",dmd_graph.number_of_nodes(),dmd_graph.number_of_edges())

In [ ]:
node2vec = Node2Vec(dmd_graph, dimensions=64, walk_length=10, num_walks=100, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [ ]:
dm_d_vec={}
for dis in disIdx:
    dm_d_vec[dis]=model.wv.get_vector(dis)

## Combine node vectors from LMS and DMS networks 

In [ ]:
X_train_lmd =[]
for ld in train:
    ld_vec = np.concatenate((lm_l_vec[ld[0]],dm_d_vec[ld[1]]))
    X_train_lmd.append(ld_vec)
X_train_lmd = np.array(X_train_lmd)

X_test_lmd =[]
for ld in test:
    ld_vec = np.concatenate((lm_l_vec[ld[0]],dm_d_vec[ld[1]]))
    X_test_lmd.append(ld_vec)
X_test_lmd = np.array(X_test_lmd)

X_train_lmd.shape, X_test_lmd.shape

In [ ]:
np.save('X_train_lmd.npy',X_train_lmd)
np.save('X_test_lmd.npy',X_test_lmd)